In [ ]:
import numpy as np
import pandas as pd
import urllib.request as ureq
import json

evic_table = pd.read_csv("data/Eviction_Notices.csv", low_memory=False)
url = "https://geo.fcc.gov/api/census/area?"

def parseLocation(ln):
    return ln[7:-1].split(" ")

In [12]:
for i, row in evic_table.iterrows():
    loc = parseLocation(str(row['Location']))
    if (len(loc) > 1):
        res = json.loads(ureq.urlopen(url + "lon=" + loc[0] + "&lat=" + loc[1]).read())
        fips = res['results'][0]['block_fips'][1:11])
        

6081600300
6075020800
6075022901
6075033203
6075015802
6075026301
6075020800


KeyboardInterrupt: 